<a href="https://colab.research.google.com/github/nigeltc/hugging-face-course/blob/main/hfc2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HFC2 - Using Transformers



In [1]:
! pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 15.3 MB/s eta 0:00:00


## Behind the Pipeline

In [2]:
from transformers import AutoTokenizer

In [3]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [4]:
raw_inputs = [
    "I've been waiting for a HuggingFace course all my life.",
    "I hate this so much."
]
inputs = tokenizer(raw_inputs,
                   padding=True,
                   truncation=True,
                   return_tensors="pt")

In [5]:
print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2035,  2026,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,  1012,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


## Going through the Model

In [6]:
from transformers import AutoModel
model = AutoModel.from_pretrained(checkpoint)

Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertModel: ['classifier.weight', 'classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
outputs = model(**inputs)

In [8]:
print(outputs.last_hidden_state.shape)

torch.Size([2, 16, 768])


## Model Heads: Making Sense of the Numbers

In [9]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [10]:
outputs = model(**inputs)

In [11]:
print(outputs.logits.shape)

torch.Size([2, 2])


## Postprocessing the Output

In [12]:
print(outputs.logits)

tensor([[-1.8599,  1.8948],
        [ 4.2321, -3.4101]], grad_fn=<AddmmBackward0>)


In [13]:
import torch
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

In [14]:
print(predictions)

tensor([[2.2873e-02, 9.7713e-01],
        [9.9952e-01, 4.7952e-04]], grad_fn=<SoftmaxBackward0>)


In [15]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

## Tokenizers

In [17]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

In [18]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [19]:
tokenizer("Using a transformer network is simple")

{'input_ids': [101, 7993, 170, 11303, 1200, 2443, 1110, 3014, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [21]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [22]:
sequence = "Using a Transformer network is simple"
tokens = tokenizer.tokenize(sequence)

In [23]:
print(tokens)

['using', 'a', 'transform', '##er', 'network', 'is', 'simple']


In [24]:
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[2478, 1037, 10938, 2121, 2897, 2003, 3722]


In [26]:
tokenizer.decode(ids)

'using a transformer network is simple'